In [1]:
from test_set_read import Read_from_TestSet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline
train_data = Read_from_TestSet()


begin to read data files,time:20:54:25
File Reading Finish,time:20:54:42


In [2]:
from collections import defaultdict 
res = defaultdict(list) 
for i, j in train_data: 
    res[i] = res[i]+ [x for x in j]
my_dict = dict(res)
#print(my_dict['8746'])
#print(dct)
print(my_dict['8746'].count('.....'))
"""df = pd.DataFrame(train_data, columns=['author', 'text'])
author = 0
text_list = []
text = {}

for index,row in df.iterrows():
    #print(row['author'])
    if row['author'] in text:
        
        
        text[row['author']].extend(row['text'])
        
        
       
    else:
        
        
        text[row['author']] = row['text']
       
print(text['8746'])"""
"""from sklearn.model_selection import train_test_split
author = df['author']
text = df['text']
X_train, X_test, Y_train, Y_test = train_test_split(author, text)
print("Training set has {} instances. Test set has {} instances.".format(X_train.shape[0], X_test.shape[0]))"""

0


'from sklearn.model_selection import train_test_split\nauthor = df[\'author\']\ntext = df[\'text\']\nX_train, X_test, Y_train, Y_test = train_test_split(author, text)\nprint("Training set has {} instances. Test set has {} instances.".format(X_train.shape[0], X_test.shape[0]))'

In [9]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
import gensim
from gensim.models.doc2vec import TaggedDocument
import re
from sklearn.model_selection import train_test_split
df = pd.read_csv('train_tweets.txt', header = None,sep='\t')
#print(df)
df.columns = ['author','text']
#df = pd.DataFrame(train_data, columns=['author', 'text'])
author = df['author']
text = df['text']

train, test = train_test_split(df[:1000], test_size=0.3, random_state=42)
import nltk
from nltk.corpus import stopwords
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens
train_tagged = train.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['text']), tags=[r.author]), axis=1)
test_tagged = test.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['text']), tags=[r.author]), axis=1)

print(train_tagged.values[0])

TaggedDocument(['handle', 'hi', 'cat', 'was', 'wondering', 'if', 'you', 'would', 'sign', 'baseball', 'card', 'through', 'the', 'mail'], [3815])


In [11]:
import multiprocessing
cores = multiprocessing.cpu_count()
print(cores)

8


In [21]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=15, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

C:\ProgramData\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "
100%|████████████████████████████████████████████████| 700/700 [00:00<?, ?it/s]


In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC


def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    
    return targets, regressors
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)
print(X_train[:2])
logreg = OneVsRestClassifier(LinearSVC())
#logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

(array([-0.03131738,  0.08722727,  0.02925882,  0.09875522,  0.08717991,
        0.10422412, -0.19410866,  0.08946384, -0.00538243, -0.12347768,
        0.03565061, -0.09805083, -0.08203804, -0.05408343, -0.02493394,
        0.03850761, -0.06846736, -0.04325011,  0.06283724,  0.0111905 ,
       -0.0565585 , -0.1260067 ,  0.01774266, -0.01563613, -0.12847003,
        0.01363741, -0.04396375,  0.09216774,  0.10215259,  0.01640328,
        0.00441635, -0.06269327,  0.02152584,  0.09396421,  0.04853328,
       -0.02837271, -0.13151592, -0.08740183, -0.10708866,  0.17553689,
       -0.05245179,  0.06429687, -0.00754783,  0.04279668,  0.04832714,
       -0.07538051,  0.08357435,  0.00620718, -0.04781749,  0.07966918,
       -0.05230391, -0.03724572,  0.16673964,  0.09876604, -0.01083407,
        0.01403367,  0.1381347 , -0.02484476,  0.00985916,  0.13829914,
        0.01385615, -0.00350267, -0.05376802,  0.05224146, -0.04685213,
       -0.02693198, -0.09306385,  0.09088743,  0.06023801, -0.0

      dtype=float32))
Testing accuracy 0.53


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Testing F1 score: 0.5125954970903428


In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import LinearSVC
#mlb = MultiLabelBinarizer()
#mlb.fit_transform(y_train)
logreg = OneVsRestClassifier(LinearSVC())
#logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(train_vectors_dbow, y_train)
logreg = logreg.fit(train_vectors_dbow, y_train)
y_pred = logreg.predict(test_vectors_dbow)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

NameError: name 'OneVsRestClassifier' is not defined